# install import

In [2]:
pip install torch torchvision torchaudio


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install prophet

  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.9 MB/s eta 0:00:00
Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install statsmodels

  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.7 MB/s eta 0:00:00 0:00:01
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 6.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 7.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-04 10:13:36.444620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# begin

In [2]:
# Load dataset
df = pd.read_csv('Dataset/Processed/BitcoinAllHistoricalPrice.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df['volume'] = pd.to_numeric(df['volume'], errors='coerce')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5457 entries, 2010-07-18 to 2025-07-01
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   5457 non-null   float64
 1   open    5457 non-null   float64
 2   high    5457 non-null   float64
 3   low     5457 non-null   float64
 4   volume  5430 non-null   float64
 5   change  5457 non-null   float64
dtypes: float64(6)
memory usage: 298.4 KB


In [4]:



# Feature engineering
df['price_change'] = df['price'].pct_change()
df['volume_change'] = df['volume'].pct_change()
df['rolling_mean_7'] = df['price'].rolling(window=7).mean()
df['rolling_std_7'] = df['price'].rolling(window=7).std()
df['rolling_mean_30'] = df['price'].rolling(window=30).mean()
df['rolling_std_30'] = df['price'].rolling(window=30).std()
df['exp_moving_avg'] = df['price'].ewm(span=30, adjust=False).mean()
df['price_diff'] = df['high'] - df['low']
df.dropna(inplace=True)

# Define features and target
features = ['open', 'high', 'low', 'volume', 'price_change', 'volume_change',
            'rolling_mean_7', 'rolling_std_7', 'rolling_mean_30', 'rolling_std_30',
            'exp_moving_avg', 'price_diff']
target = 'price'

X = df[features]
y = df[target]

# Time Series Cross Validation
ts_cv = TimeSeriesSplit(n_splits=5)




/var/folders/jg/syzw72_n52x0x5b9sggw57x80000gn/T/ipykernel_79876/3760880242.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['volume_change'] = df['volume'].pct_change()


In [5]:
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(),
    'KNN': KNeighborsRegressor(),
    'DecisionTree': DecisionTreeRegressor(),
    'XGBoost': XGBRegressor(),
    'CatBoost': CatBoostRegressor(verbose=0),
    'SVR': SVR()
}

model_scores = {}

for name, model in models.items():
    r2_scores = []
    mae_scores = []
    rmse_scores = []

    for train_idx, test_idx in ts_cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        r2_scores.append(r2_score(y_test, preds))
        mae_scores.append(mean_absolute_error(y_test, preds))
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, preds)))

    model_scores[name] = {
        'R2': np.mean(r2_scores),
        'MAE': np.mean(mae_scores),
        'RMSE': np.mean(rmse_scores)
    }


In [ ]:

# Transformer Model
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_size, d_model=64, nhead=4, num_layers=2, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = self.fc_out(x.mean(dim=1))
        return x

# Preprocess for Transformer
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

window_size = 30
X_seq = []
y_seq = []

for i in range(len(X_tensor) - window_size):
    X_seq.append(X_tensor[i:i+window_size])
    y_seq.append(y_tensor[i+window_size])

X_seq = torch.stack(X_seq)
y_seq = torch.stack(y_seq)

transformer_model = TimeSeriesTransformer(input_size=X_seq.shape[2])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=0.001)

# Train Transformer
epochs = 20
for epoch in range(epochs):
    transformer_model.train()
    optimizer.zero_grad()
    output = transformer_model(X_seq)
    loss = criterion(output, y_seq)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 5 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

print("\nTime Series Cross Validation Results:")
for model_name, scores in model_scores.items():
    print(f"{model_name} -> R²: {scores['R2']:.4f}, MAE: {scores['MAE']:.4f}, RMSE: {scores['RMSE']:.4f}")


/usr/local/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:


# Visualization
def plot_predictions(y_true, y_pred, title):
    plt.figure(figsize=(12, 6))
    plt.plot(y_true, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Bitcoin Price')
    plt.legend()
    plt.show()

for name, model in models.items():
    X_train, X_test = X.iloc[:-100], X.iloc[-100:]
    y_train, y_test = y.iloc[:-100], y.iloc[-100:]

    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    plot_predictions(y_test.values, preds, f'{name} Predictions')

# LSTM Model
scaler_lstm = MinMaxScaler()
X_scaled_lstm = scaler_lstm.fit_transform(X)

generator = TimeseriesGenerator(X_scaled_lstm, y.values, length=30, batch_size=32)

lstm_model = Sequential()
lstm_model.add(LSTM(64, activation='relu', input_shape=(30, X.shape[1]), return_sequences=True))
lstm_model.add(LSTM(32, activation='relu'))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit(generator, epochs=20)

# CNN + LSTM Model
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(30, X.shape[1])))
cnn_lstm_model.add(MaxPooling1D(pool_size=2))
cnn_lstm_model.add(LSTM(50, activation='relu'))
cnn_lstm_model.add(Dense(1))
cnn_lstm_model.compile(optimizer='adam', loss='mse')

cnn_lstm_model.fit(generator, epochs=20)

# Prophet Model
prophet_df = df[['price']].reset_index().rename(columns={'date': 'ds', 'price': 'y'})
prophet_model = Prophet()
prophet_model.fit(prophet_df)

future = prophet_model.make_future_dataframe(periods=100)
forecast = prophet_model.predict(future)

prophet_model.plot(forecast)
plt.title('Prophet Forecast')
plt.show()

# Summary print
print("\nAll models have been trained and evaluated. Visualization completed.")

In [ ]:
# Prophet Specific Prediction
besok = forecast.iloc[-100]['yhat']
minggu_depan = forecast.iloc[-93]['yhat']
bulan_depan = forecast.iloc[-70]['yhat']
tahun_depan = forecast.iloc[-1]['yhat']

print(f"Prediksi Besok: {besok}")
print(f"Prediksi Minggu Depan: {minggu_depan}")
print(f"Prediksi Bulan Depan: {bulan_depan}")
print(f"Prediksi Tahun Depan: {tahun_depan}")
